In [31]:
import os, cv2
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import keras
from collections import defaultdict, namedtuple
from tqdm import tqdm
from tensorflow.keras.applications import Xception
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.applications import xception, vgg19
from tensorflow.keras.applications import Xception, vgg19
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input, GlobalAveragePooling2D, Activation, Dense, Dropout 
from tensorflow.keras.layers import BatchNormalization, Conv2D, Embedding, MaxPool2D, SeparableConv2D, MaxPooling2D, concatenate, UpSampling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import layers
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.applications import Xception,resnet 
from tensorflow.keras import regularizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import *
from keras.optimizers import Adam, SGD, Adamax
from keras.layers import Concatenate
from keras.utils.np_utils import to_categorical 

# gpu 사용가능 확인

In [32]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15624175166737760826
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
locality {
  bus_id: 1
}
incarnation: 6775805986659839279
physical_device_desc: "device: 0, name: METAL, pci bus id: <undefined>"
xla_global_id: -1
]


2022-07-17 01:52:16.852454: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-17 01:52:16.852569: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# 필요한 만큼 메모리를 런타임에 할당하는 방법

In [33]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)     


1 Physical GPUs, 1 Logical GPUs


# warnings hide

In [34]:
import warnings
# 경고 메시지를 무시하고 숨기거나
warnings.filterwarnings(action='ignore')

In [35]:
SEED = 42
seed = 42
EPOCHS = 100
epochs = 100
BATCH_SIZE = 32
batch_size = 32
IMG_HEIGHT = 224
img_height = 224
IMG_WIDTH = 224
img_width = 224
num_classes = 8
patience = 5

In [36]:
def seed_everything(seed):
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

In [37]:
seed_everything(seed)

In [38]:
full_df = pd.read_csv('full_df(agumentation).csv', index_col= 0)

In [39]:
full_df

,filename,labels
1234,preprocessed_images(divide)/N/2499_right.jpg,N
13560,preprocessed_images(divide)/D/4129_right.jpg,D
10683,preprocessed_images(divide)/C/2118_right.jpg,C
12183,preprocessed_images(divide)/C/C_180_2189_left.jpg,C
6400,preprocessed_images(divide)/O/O_270_52_right.jpg,O
...,...,...
13095,preprocessed_images(divide)/D/280_right.jpg,D
10842,preprocessed_images(divide)/C/C_blur_2137_righ...,C
616,preprocessed_images(divide)/N/2106_left.jpg,N
9169,preprocessed_images(divide)/M/106_right.jpg,M


In [40]:
full_df['labels'] = full_df['labels'].map({'N' : 0, 'D' : 1, 'G' : 2, 'C' : 3,  'A' : 4, 'H' : 5, 'M' : 6, 'O' : 7})

In [41]:
full_df

,filename,labels
1234,preprocessed_images(divide)/N/2499_right.jpg,0
13560,preprocessed_images(divide)/D/4129_right.jpg,1
10683,preprocessed_images(divide)/C/2118_right.jpg,3
12183,preprocessed_images(divide)/C/C_180_2189_left.jpg,3
6400,preprocessed_images(divide)/O/O_270_52_right.jpg,7
...,...,...
13095,preprocessed_images(divide)/D/280_right.jpg,1
10842,preprocessed_images(divide)/C/C_blur_2137_righ...,3
616,preprocessed_images(divide)/N/2106_left.jpg,0
9169,preprocessed_images(divide)/M/106_right.jpg,6


In [42]:
full_df['labels'].value_counts()

0    2873
7    2124
1    2008
3    1758
2    1704
4    1596
6    1392
5     768
Name: labels, dtype: int64

In [43]:
full_df['labels'].value_counts().sum()

14223

In [44]:
df

,filename,labels
1234,preprocessed_images(divide)/N/2499_right.jpg,0
13560,preprocessed_images(divide)/D/4129_right.jpg,1
10683,preprocessed_images(divide)/C/2118_right.jpg,3
12183,preprocessed_images(divide)/C/C_180_2189_left.jpg,3
6400,preprocessed_images(divide)/O/O_270_52_right.jpg,7
...,...,...
13095,preprocessed_images(divide)/D/280_right.jpg,1
10842,preprocessed_images(divide)/C/C_blur_2137_righ...,3
616,preprocessed_images(divide)/N/2106_left.jpg,0
9169,preprocessed_images(divide)/M/106_right.jpg,6


In [45]:
df = full_df

# 데이터

In [46]:
def create_datasets(df, img_width, img_height):
    imgs = []
    for path in tqdm(df['filename']):
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (img_width, img_height))
        imgs.append(img)
        
    imgs = np.array(imgs, dtype='float32')
#   df = pd.get_dummies(df['labels'])
    df = to_categorical(df['labels'], num_classes = 8)
    return imgs, df

In [47]:
# train data, test data 분리
train_df, test_df = train_test_split(df,
                                     test_size=0.30, 
                                     random_state=42,
                                     stratify=df['labels']
                                    )

train_df, val_df = train_test_split(train_df,
                                    test_size=0.30,
                                    random_state=42,
                                    stratify=train_df['labels']
                                   )

In [48]:
train_df.shape, test_df.shape, train_df.shape, val_df.shape

((6969, 2), (4267, 2), (6969, 2), (2987, 2))

In [49]:
train_imgs, train_df = create_datasets(train_df, IMG_WIDTH, IMG_HEIGHT)
val_imgs, val_df = create_datasets(val_df, IMG_WIDTH, IMG_HEIGHT)
test_imgs, test_df = create_datasets(test_df, IMG_WIDTH, IMG_HEIGHT)

100%|██████████████████████████████████████| 4267/4267 [00:10<00:00, 392.43it/s]


In [50]:
train_imgs = train_imgs / 255.0
val_imgs = val_imgs / 255.0
test_imgs = test_imgs / 255.0

In [51]:
len(train_imgs), len(val_imgs), len(test_imgs)

(6969, 2987, 4267)

In [52]:
train_imgs.shape, val_imgs.shape, test_imgs.shape

((6969, 224, 224, 3), (2987, 224, 224, 3), (4267, 224, 224, 3))

In [53]:
train_df.shape, val_df.shape, test_df.shape

((6969, 8), (2987, 8), (4267, 8))

In [54]:
'''
train_imgs = preprocess_input(train_imgs)
val_imgs = preprocess_input(val_imgs)
test_imgs = preprocess_input(test_imgs)
'''

'\ntrain_imgs = preprocess_input(train_imgs)\nval_imgs = preprocess_input(val_imgs)\ntest_imgs = preprocess_input(test_imgs)\n'

# MODEL

In [55]:
BinaryAccuracy = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
]

In [56]:
del model

NameError: name 'model' is not defined

In [57]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Activation, Flatten, Dropout, Dense


class Classifier:
    def build(width, height, depth):
        model = Sequential()
        img_shape = (224, 224, 3)
        model.add(Conv2D(32, (3, 3), padding='same', input_shape=img_shape))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(3, 3)))
        model.add(Dropout(0.25))

        model.add(Conv2D(64, (3, 3), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=-1))
        model.add(Dropout(0.5))
        model.add(Conv2D(64, (3, 3), padding='same'))
        model.add(Activation('relu'))

        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))
        model.add(Conv2D(128, (3, 3), padding='same'))
        model.add(Activation('relu'))

        model.add(Dropout(0.5))
        model.add(Conv2D(128, (3, 3), padding='same'))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))

        model.add(Conv2D(256, (3, 3), padding='same'))
        model.add(Activation('relu'))
        model.add(Dropout(0.5))
        model.add(Conv2D(256, (3, 3), padding='same'))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))

        model.add(Flatten())
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        model.add(Dense(8))
        model.add(Activation('sigmoid'))

        sgd = SGD(learning_rate=0.0001)
        model.compile(optimizer=sgd, loss='mean_squared_error', metrics=[BinaryAccuracy])   #binary_crossentropy
        return model    

In [60]:
model = Classifier.build(224, 224, 3)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 activation (Activation)     (None, 224, 224, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 74, 74, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 74, 74, 64)        18496     
                                                                 
 activation_1 (Activation)   (None, 74, 74, 64)        0         
                                                        

In [61]:
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, 
                                               verbose=1, 
                                               restore_best_weights=True)


reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=10, verbose=1)

In [62]:
history = model.fit(train_imgs, 
                    train_df,
                    batch_size=16, 
                    epochs=50,
                    #steps_per_epoch=len(train_imgs)/BATCH_SIZE,
                    callbacks=[es_callback, reduce_lr],
                    validation_data=(val_imgs, val_df))


pd.DataFrame(history.history)[['accuracy', 'val_accuracy']].plot()
pd.DataFrame(history.history)[['loss', 'val_loss']].plot()
plt.show()

Epoch 1/50


2022-07-17 01:53:54.301993: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-17 01:53:55.227340: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


436/436 [==============================] - ETA: 0s - loss: 0.2471 - accuracy: 0.8750

2022-07-17 01:55:11.274772: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


436/436 [==============================] - 86s 196ms/step - loss: 0.2471 - accuracy: 0.8750 - val_loss: 0.2029 - val_accuracy: 0.6999 - lr: 1.0000e-04
Epoch 2/50
436/436 [==============================] - 81s 185ms/step - loss: 0.2413 - accuracy: 0.8750 - val_loss: 0.2901 - val_accuracy: 0.6979 - lr: 1.0000e-04
Epoch 3/50
436/436 [==============================] - 80s 184ms/step - loss: 0.2358 - accuracy: 0.8750 - val_loss: 0.2917 - val_accuracy: 0.6981 - lr: 1.0000e-04
Epoch 4/50
436/436 [==============================] - 83s 189ms/step - loss: 0.2304 - accuracy: 0.8750 - val_loss: 0.2912 - val_accuracy: 0.6985 - lr: 1.0000e-04
Epoch 5/50
436/436 [==============================] - 83s 190ms/step - loss: 0.2252 - accuracy: 0.8750 - val_loss: 0.2906 - val_accuracy: 0.6990 - lr: 1.0000e-04
Epoch 6/50
436/436 [==============================] - 88s 201ms/step - loss: 0.2203 - accuracy: 0.8750 - val_loss: 0.2899 - val_accuracy: 0.6994 - lr: 1.0000e-04
Epoch 7/50
436/436 [===================


KeyboardInterrupt



In [63]:
model.predict(test_imgs)[1]

2022-07-17 02:23:00.670179: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


134/134 [==============================] - 15s 103ms/step


array([1.33079433e-11, 1.51104864e-03, 3.10314761e-08, 5.55628286e-11,
       9.48554158e-01, 1.27463925e-11, 3.13038084e-09, 1.00000000e+00],
      dtype=float32)

# Inception_ResnetV2

In [163]:
del model

In [164]:
del base_model

In [165]:
del x

In [166]:
BinaryAccuracy = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
]

In [168]:
base_model=tf.keras.applications.EfficientNetB0(include_top=False,input_shape=(224, 224, 3)) 

base_model.trainable = True

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.25)(x)



output= Dense(8, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=output)
model.summary()

Model: "model_123"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_5 (Rescaling)        (None, 224, 224, 3)  0           ['input_6[0][0]']                
                                                                                                  
 normalization_5 (Normalization  (None, 224, 224, 3)  7          ['rescaling_5[0][0]']            
 )                                                                                                
                                                                                          

                                                                                                  
 block2a_project_conv (Conv2D)  (None, 56, 56, 24)   2304        ['block2a_se_excite[0][0]']      
                                                                                                  
 block2a_project_bn (BatchNorma  (None, 56, 56, 24)  96          ['block2a_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block2b_expand_conv (Conv2D)   (None, 56, 56, 144)  3456        ['block2a_project_bn[0][0]']     
                                                                                                  
 block2b_expand_bn (BatchNormal  (None, 56, 56, 144)  576        ['block2b_expand_conv[0][0]']    
 ization)                                                                                         
          

                                                                                                  
 block3b_expand_bn (BatchNormal  (None, 28, 28, 240)  960        ['block3b_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block3b_expand_activation (Act  (None, 28, 28, 240)  0          ['block3b_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block3b_dwconv (DepthwiseConv2  (None, 28, 28, 240)  6000       ['block3b_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block3b_b

 block4b_dwconv (DepthwiseConv2  (None, 14, 14, 480)  4320       ['block4b_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block4b_bn (BatchNormalization  (None, 14, 14, 480)  1920       ['block4b_dwconv[0][0]']         
 )                                                                                                
                                                                                                  
 block4b_activation (Activation  (None, 14, 14, 480)  0          ['block4b_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block4b_se_squeeze (GlobalAver  (None, 480)         0           ['block4b_activation[0][0]']     
 agePoolin

                                                                                                  
 block5a_activation (Activation  (None, 14, 14, 480)  0          ['block5a_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block5a_se_squeeze (GlobalAver  (None, 480)         0           ['block5a_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block5a_se_reshape (Reshape)   (None, 1, 1, 480)    0           ['block5a_se_squeeze[0][0]']     
                                                                                                  
 block5a_se_reduce (Conv2D)     (None, 1, 1, 20)     9620        ['block5a_se_reshape[0][0]']     
          

                                                                                                  
 block5c_se_expand (Conv2D)     (None, 1, 1, 672)    19488       ['block5c_se_reduce[0][0]']      
                                                                                                  
 block5c_se_excite (Multiply)   (None, 14, 14, 672)  0           ['block5c_activation[0][0]',     
                                                                  'block5c_se_expand[0][0]']      
                                                                                                  
 block5c_project_conv (Conv2D)  (None, 14, 14, 112)  75264       ['block5c_se_excite[0][0]']      
                                                                                                  
 block5c_project_bn (BatchNorma  (None, 14, 14, 112)  448        ['block5c_project_conv[0][0]']   
 lization)                                                                                        
          

                                                                                                  
 block6b_project_bn (BatchNorma  (None, 7, 7, 192)   768         ['block6b_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block6b_drop (Dropout)         (None, 7, 7, 192)    0           ['block6b_project_bn[0][0]']     
                                                                                                  
 block6b_add (Add)              (None, 7, 7, 192)    0           ['block6b_drop[0][0]',           
                                                                  'block6a_project_bn[0][0]']     
                                                                                                  
 block6c_expand_conv (Conv2D)   (None, 7, 7, 1152)   221184      ['block6b_add[0][0]']            
          

                                                                                                  
 block6d_add (Add)              (None, 7, 7, 192)    0           ['block6d_drop[0][0]',           
                                                                  'block6c_add[0][0]']            
                                                                                                  
 block7a_expand_conv (Conv2D)   (None, 7, 7, 1152)   221184      ['block6d_add[0][0]']            
                                                                                                  
 block7a_expand_bn (BatchNormal  (None, 7, 7, 1152)  4608        ['block7a_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block7a_expand_activation (Act  (None, 7, 7, 1152)  0           ['block7a_expand_bn[0][0]']      
 ivation) 

In [169]:
sgd = SGD(learning_rate=0.001)
model.compile(optimizer = sgd, loss = 'mean_squared_error', metrics = [BinaryAccuracy]) 

In [170]:
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, 
                                               verbose=1, 
                                               restore_best_weights=True)


reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=10, verbose=1)

In [171]:
history = model.fit(train_imgs, train_df,
                    epochs=epochs,
                    batch_size=batch_size,
                    shuffle=True, 
                    verbose = 1,
                    validation_data=(val_imgs, val_df), 
                    callbacks=[es_callback, reduce_lr]     
                   )



pd.DataFrame(history.history)[['accuracy', 'val_accuracy']].plot()
pd.DataFrame(history.history)[['loss', 'val_loss']].plot()
plt.show()

Epoch 1/100


2022-07-17 01:28:35.813030: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


200/200 [==============================] - ETA: 0s - loss: 0.1095 - accuracy: 0.8750

2022-07-17 01:32:36.668464: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


200/200 [==============================] - 262s 1s/step - loss: 0.1095 - accuracy: 0.8750 - val_loss: 0.1109 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 2/100
200/200 [==============================] - 249s 1s/step - loss: 0.1091 - accuracy: 0.8750 - val_loss: 0.1197 - val_accuracy: 0.8630 - lr: 0.0010
Epoch 3/100
200/200 [==============================] - 250s 1s/step - loss: 0.1090 - accuracy: 0.8750 - val_loss: 0.2158 - val_accuracy: 0.7839 - lr: 0.0010
Epoch 4/100
200/200 [==============================] - 249s 1s/step - loss: 0.1089 - accuracy: 0.8750 - val_loss: 0.2192 - val_accuracy: 0.7806 - lr: 0.0010
Epoch 5/100
164/200 [=======================>......] - ETA: 43s - loss: 0.1087 - accuracy: 0.8750


KeyboardInterrupt

